In [9]:
import pandas as pd
import numpy as np
import time
from p_tqdm import p_map
import geopandas as gpd

import sqlite3
sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))

In [1]:
from const import DATASETS,SETTINGS


TYPE310 Output_Areas 2001
TYPE299 Output_Areas 2011
TYPE304 Lower_Layer_Super_Output_Areas 2001
TYPE298 Lower_Layer_Super_Output_Areas 2011
TYPE305 Middle_Layer_Super_Output_Areas 2001
TYPE297 Middle_Layer_Super_Output_Areas 2011


### Merge all centroids for relevant Layer. 

In [6]:
# def get_centroids(kind):
areatype = 'Output_Areas'    

__outdir__='flatdb/'+areatype
os.mkdir(__outdir__)




### Load the largeAreas (county/UA)

OA should not change much


In [10]:
gpd.read_file('Counties_and_Unitary_Authorities_(December_2020)_UK_BGC.geojson')

,OBJECTID,CTYUA20CD,CTYUA20NM,CTYUA20NMW,BNG_E,BNG_N,LONG,LAT,Shape__Area,Shape__Length,geometry
0,1,E06000001,Hartlepool,,447160,531474,-1.27018,54.67614,9.377020e+07,68482.701053,"POLYGON ((-1.27025 54.72717, -1.26822 54.72609..."
1,2,E06000002,Middlesbrough,,451141,516887,-1.21099,54.54467,5.385812e+07,42570.770613,"MULTIPOLYGON (((-1.19752 54.58211, -1.19434 54..."
2,3,E06000003,Redcar and Cleveland,,464361,519597,-1.00608,54.56752,2.451392e+08,94691.669222,"MULTIPOLYGON (((-1.20098 54.57763, -1.20030 54..."
3,4,E06000004,Stockton-on-Tees,,444940,518183,-1.30664,54.55691,2.049030e+08,118321.073248,"MULTIPOLYGON (((-1.27113 54.55427, -1.27213 54..."
4,5,E06000005,Darlington,,428029,515648,-1.56835,54.53534,1.974858e+08,105777.871675,"POLYGON ((-1.63800 54.61720, -1.63767 54.61670..."
...,...,...,...,...,...,...,...,...,...,...,...
211,212,W06000020,Torfaen,Torfaen,327459,200480,-3.05101,51.69836,1.262231e+08,78980.290566,"POLYGON ((-3.10597 51.79548, -3.09636 51.79155..."
212,213,W06000021,Monmouthshire,Sir Fynwy,337812,209231,-2.90280,51.77827,8.503214e+08,218882.426426,"MULTIPOLYGON (((-2.78092 51.52605, -2.78124 51..."
213,214,W06000022,Newport,Casnewydd,337897,187432,-2.89769,51.58231,1.904458e+08,148229.838483,"MULTIPOLYGON (((-2.94668 51.62718, -2.94672 51..."
214,215,W06000023,Powys,Powys,302329,273255,-3.43531,52.34864,5.195418e+09,589999.998341,"MULTIPOLYGON (((-3.90623 52.56080, -3.90861 52..."


## GENERATE CENTROIDS AND NAME ARRAY

In [5]:
cdf = pd.DataFrame()
for y in SETTINGS.years:
    f = SETTINGS.matchC(areatype,y)
    print(f)
    dummy = pd.read_csv(f)
    dummy['YEAR'] = y
    # dummy.columns = ['AREACD' if 'cd' in x.lower() else x for x in dummy.columns]
    cdf = pd.concat([cdf, dummy ],axis=0)# df.append(process_df(url,src))

cdf['YEAR'] = 2011

cdf.columns = 'AREACD LAT LON YEAR'.split()
cdf.groupby('AREACD').mean().reset_index()
cdf['GEOGRAPHY_CODE'] = cdf.index
cdf = cdf.astype({'AREACD': str,
            'LAT': np.int32,
            'LON': np.int32,
            'YEAR': np.int32,
            'GEOGRAPHY_CODE': np.int32})

geomap = dict(zip(cdf.AREACD,cdf.GEOGRAPHY_CODE))
cdf



db_file = f'{__outdir__}/centroids.sqlite'

conn = sqlite3.connect(db_file)

./centroids/WGS84_simplified/Output_Areas__December_2011__Population_Weighted_Centroids.csv
./centroids/WGS84_simplified/Output_Areas__December_2001__Population_Weighted_Centroids.csv


,AREACD,LAT,LON,YEAR,GEOGRAPHY_CODE
0,E00096400,4989727,-634638,2011,0
1,E00096408,4995545,-633715,2011,1
2,E00096403,4991444,-631716,2011,2
3,E00096407,4991486,-631691,2011,3
4,E00096405,4991445,-631222,2011,4
...,...,...,...,...,...
175429,E00154294,5248528,175409,2011,175429
175430,E00154282,5248180,175456,2011,175430
175431,E00154272,5248360,175540,2011,175431
175432,E00154271,5248639,175580,2011,175432


##  A function to get the individual dataframes

In [16]:
def process_df(url, src):
    '''Read each individaul query and return a dataframe'''
    print(url)
    dummydf = pd.read_csv(url)
    dummydf.columns = 'GEOGRAPHY_CODE YEAR CELL OBS_VALUE'.split()

    if 'cellmaps' in src:
        cells = '0,'+','.join(src['cellnames'])
        cells = ','.join(src['cellnames'])
        for v, k in src['cellmaps'].items(): 
            for c in k: 
                cells = cells.replace(c,v)
        src['cellnames'] = ('Totals,'+cells).split(',')

    mapping = dict(zip(eval(src['cells']), src['cellnames']))
    dummydf['CELL'] = dummydf['CELL'].apply(lambda x: mapping[x])   # (rename,mapping = mapping)
    return dummydf



## Process each Year and concat

In [23]:
def fetch(dataset):
    '''Fetch the data from the source'''
    global SETTINGS
    time.sleep(np.random.rand()*10.)
    sources = dataset['sources']
    #     src = sources[0].copy()
    #      dont use pointers copy

    df = False
    
    for src in sources:
        src = src.copy()
        src['cells'] = ','.join(map(str, src['cells']))
        src['selection'] = ','.join(map(str, src['selection']))
        src['key'] = SETTINGS.conf['apikey']
        src['geocode'] = SETTINGS.geocode[areatype][src['year']]
        if not hasattr(src, 'query_misc'): src['query_misc'] = ''

        url = SETTINGS.conf['apiurl'].format(**src)
        if type(df) != bool: df = pd.concat([df, process_df(url, src)]) # df.append(process_df(url,src))
        else: df = process_df(url, src)
        print(df.tail(2))

    # return df
    print('mapping')
    # subcategory = mapping
    df = df[['GEOGRAPHY_CODE', 'CELL', 'OBS_VALUE','YEAR']]
    df = df.sort_values('CELL')
    order = df.CELL.drop_duplicates().to_list()


    if 'Totals' not in order: 
        b = df.groupby('GEOGRAPHY_CODE').sum()
        b.CELL = 'Totals'
        df = pd.concat([df,b.reset_index()],axis=0)  
        order=['Totals']+order

    mapping = dict(zip(order, range(len(order))))
    
    print(order)

    # datamong
#  drop novalues

    df = df[df.GEOGRAPHY_CODE.isin(geomap.keys())]
    df['GEOGRAPHY_CODE'] = df['GEOGRAPHY_CODE'].apply(lambda x: geomap[x]).astype(np.uint16)
    i = df.groupby(['GEOGRAPHY_CODE', 'CELL']).sum().astype(np.uint32).reset_index()
    i.CELL = [mapping[k] for k in i.CELL]
    i = i[i.OBS_VALUE > 0] # remove zero values

    # return i[['GEOGRAPHY_CODE', 'CELL', 'OBS_VALUE']]]
    

    return [dataset['code'], order , i[['GEOGRAPHY_CODE', 'CELL', 'OBS_VALUE']]]


In [24]:
SETTINGS.geocode[areatype]

{'2001': 'TYPE310', '2011': 'TYPE299'}

### Parallel Fetch 


In [25]:

# results = p_map(fetch,DATASETS)

results = []
for i in DATASETS:
    results.append(fetch(i))

# print(results)

# a = fetch(DATASETS[4])

http://www.nomisweb.co.uk/api/v01/dataset/NM_1634_1.data.csv?date=latest&geography=TYPE310&cell=1,2&select=geography_code,date_name,cell,obs_value&uid=0xc85f0574351e4660fa95d5e18ec9c4bac91026c3
       GEOGRAPHY_CODE  YEAR    CELL  OBS_VALUE
701734      W00002267  2001  female      134.0
701735      W00002267  2001  female       50.6
http://www.nomisweb.co.uk/api/v01/dataset/NM_144_1.data.csv?date=latest&geography=TYPE299&cell=1,2&select=geography_code,date_name,cell,obs_value&uid=0xc85f0574351e4660fa95d5e18ec9c4bac91026c3
       GEOGRAPHY_CODE  YEAR    CELL  OBS_VALUE
999998      E00028054  2011  female        0.0
999999      E00028054  2011  female        NaN
mapping
['Totals', 'female', 'male']
http://www.nomisweb.co.uk/api/v01/dataset/NM_1637_1.data.csv?date=latest&geography=TYPE310&c_age=1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69

In [26]:


# cur = conn.cursor()


for name,order,df in results:
    print(name)
    order = pd.DataFrame(order, columns = ['NAME'])
    # order['CELL'] = order.index
    # print(order)
    df.to_sql(name, con=conn, if_exists='replace',index_label='GEOMETRY_ID')
    order.to_sql(name+'_names', con=conn, if_exists='replace',index_label='CELL')



population
age
ethnicity
health
economic
travel
tenure


### Write To DB

### SHRINK THE DATABASE

In [27]:
from pathlib import Path
file_size = Path(db_file).stat().st_size

print("The file size WAS:", file_size/1048576,"MB")

conn.execute("VACUUM") ### SHRINK!!!!
conn.close()

file_size = Path(db_file).stat().st_size

print("The file size is:", file_size/1048576,"MB")

The file size WAS: 45.29296875 MB
The file size is: 42.953125 MB


### CENTROIDS table

In [ ]:
# a = results[1]
# print(a)
# print(a[a.YEAR==2011])
# # [geomap[i] for i in a.GEOGRAPHY_CODE]

# geomap
# e=a[:].GEOGRAPHY_CODE.drop_duplicates()
# print(e)
# s = 0 
# f = 0
# miss = []
# for i in e.values:

#     try:
#         (geomap[i])
#         s+=1
#     except:
#         f+=1
#         miss.append(i)
# print(s,f,len(miss))
        
# print(miss[:20])
# geomap